In [1]:
import pandas as pd
import geopandas as gpd

In [20]:
txt = "Data/Traverse.txt"
df = pd.read_csv(txt, sep = "\t")

In [21]:
df.nunique()
df.drop(columns = ['desc', 'kml_folder', 'type'], inplace = True)

df.rename(columns={'name': 'description'}, inplace=True)
df['description'].ffill(inplace=True)


In [44]:
gdf = gpd.GeoDataFrame(data = df,
                        crs = "EPSG:4326",
                        geometry = gpd.points_from_xy(df['longitude'], df['latitude'])) # type: ignore

gdf = gdf.to_crs('EPSG:32737')                        

gdf.drop(columns = ['latitude', 'longitude'], inplace = True)
gdf_ = gdf[['altitude (m)', 'description']]

In [ ]:
e = 1618010.606367111
f = -1741978.211608887

a = 0.9916765364496314
d = 1.175648448821448

b = -0.1635616585599564
c = 0.02551828620198648

# Transform to corresponding coordinates
gdf2 = gpd.GeoDataFrame((gdf.affine_transform([a, b, c, d, e, f])))
gdf2.rename({0 : 'geometry'}, axis = 1, inplace = True)

gdf2.head()

In [94]:
gdf3 = pd.concat([gdf_, gdf2], axis = 1)
gdf3 = gpd.GeoDataFrame(gdf3, crs = "EPSG:32737")
# gdf3.head(30)

In [ ]:
df_ = gdf3
df_['x'] = gdf3.geometry.x
df_['y'] = gdf3.geometry.y
df_.drop("geometry", axis = 1, inplace = True)
df_['P'] = df_.index + 1

names ={
    "description" : "D",
    "altitude (m)" : "Z",
    "x" : "E",
    "y" : "N"
    }

df_.rename(columns = names, inplace = True)
df_ = df_[['P', 'N', 'E', 'Z', 'D']]
# df_

In [97]:
export = 'C:/Users/LUNA/Desktop/Traverse.csv'
df_.to_csv(export, index=False, header=False)
